# **ME 575 - Assignment #2 Gradients, Hessians, and a Line Search ALgorithm**

---
### **1.1** 
Calculate by hand the gradient and the Hessian for each of the three functions at the specified point. You must show your work for credit (scan or include an image in your report).  
  
(a) $x_0 = [-2,2]^T, f(x_1, x_2) = x_1^2 + x_2^2$  
(b) $x_0 = [4,2]^T, f(x_1,x_2) = (x_1 - 4)^2 + (x_2 - 2)^3$  
(c) $x_0 = [1,2,-1]^T, f(x_1, x_2, x_3) = \sin(x_1) + x_2x_3^2$

In [ ]:
'''Calculate the gradient & Hessian of each of the three functions
(a) x_0 = [-2,2]^T, f(x_1, x_2) = x_1^2 + x_2^2
(b) x_0 = [4,2]^T, f(x_1, x_2) = (x_1 - 4)^2 + (x_2 - 2)^3
(x) x_0 = [1,2,-1]^T, f(x_1, x_2, x_3) = sin(x_1) + x_2 * x_3^2
'''

import numpy as np

# (a) f(x_1, x_2) = x_1^2 + x_2^2 at x0 = [-2, 2]^T
def func_a(x): 
    x1, x2 = x

    # Gradient: [2*x1, 2*x2]
    grad = np.array([2*x1, 2*x2])

    # Hessian: [[2, 0], [0, 2]]
    hess = np.array([[2, 0], [0, 2]])

    return grad, hess

# (b) f(x_1, x_2) = (x_1 - 4)^2 + (x_2 - 2)^3 at x0 = [4, 2]^T
def func_b(x):
    x1, x2 = x

    # Gradient: [2*(x1 - 4), 3*(x2 - 2)^2]
    grad = np.array([2*(x1 - 4), 3*(x2 - 2)**2])

    # Hessian: [[2, 0], [0, 6*(x2 - 2)]]
    hess = np.array([[2, 0], [0, 6*(x2 - 2)]])

    return grad, hess

# (c) f(x_1, x_2, x_3) = sin(x_1) + x_2 * x_3^2 at x0 = [1, 2, -1]^T
def func_c(x):
    x1, x2, x3 = x

    # Gradient: [cos(x1), x3^2, 2*x2*x3]
    grad = np.array([np.cos(x1), x3**2, 2*x2*x3])

    # Hessian: 
    # d2f/dx1^2 = -sin(x1), d2f/dx2^2 = 0, d2f/dx3^2 = 2*x2
    # d2f/dx2dx3 = 2*x3 
    # [[-sin(x1), 0, 0], [0, 0, 2*x3], [0, 2*x2, 2]]
    hess = np.array([[-np.sin(x1), 0, 0],
                     [0, 0, 2*x3],
                     [0, 2*x2, 2]])

    return grad, hess

# Evaluation
points = {
    'a': (np.array([-2.0, 2.0]), func_a),
    'b': (np.array([4.0, 2.0]), func_b),
    'c': (np.array([1.0, 2.0, -1.0]), func_c)
}

for label, (point, func) in points.items():
    grad, hess = func(point)
    print(f"Function {label}:")
    print(f"  Gradient at {point} = {grad}")
    print(f"  Hessian at {point} =\n{hess}\n")


Function a:
  Gradient at [-2.  2.] = [-4.  4.]
  Hessian at [-2.  2.] =
[[2 0]
 [0 2]]

Function b:
  Gradient at [4. 2.] = [0. 0.]
  Hessian at [4. 2.] =
[[2. 0.]
 [0. 0.]]

Function c:
  Gradient at [ 1.  2. -1.] = [ 0.54030231  1.         -4.        ]
  Hessian at [ 1.  2. -1.] =
[[-0.84147098  0.          0.        ]
 [ 0.          0.         -2.        ]
 [ 0.          4.          2.        ]]



---
### **1.2**  
Calculate the directional derivative AND the directional Hessian, i.e., curvature, along the direction $p = [1,-2]^T$ for each of the above three functions at $x_0$. (Use $p = [1,-2,0]^T$ for question 1.1(c) above). Show your work.  

In [4]:
'''    Calculate the direction derivative AND the directional hessian 
        of each of the three functions along the direction p = [1,-2]^T
        at x_0. (for c use p = [1, -2, 0]^T)
(a) x_0 = [-2,2]^T, f(x_1, x_2) = x_1^2 + x_2^2
(b) x_0 = [4,2]^T, f(x_1, x_2) = (x_1 - 4)^2 + (x_2 - 2)^3
(x) x_0 = [1,2,-1]^T, f(x_1, x_2, x_3) = sin(x_1) + x_2 * x_3^2
'''

import numpy as np
from scipy.optimize import approx_fprime
from scipy.optimize._numdiff import approx_derivative

def compute_directional_results(f, x0, p, label):
    # Step 1: Numerical Gradient
    epsilon = 1e-7
    grad = approx_fprime(x0, f, epsilon)
    
    # Step 2: Numerical Hessian
    # Differentiating the gradient function gives the Hessian matrix
    def grad_func(x):
        return approx_fprime(x, f, epsilon)
    hessian = approx_derivative(grad_func, x0)
    
    # Step 3: Directional calculations
    directional_derivative = np.dot(grad, p)
    directional_hessian = np.dot(p.T, np.dot(hessian, p))
    
    print(f"--- {label} ---")
    print(f"Directional Derivative: {directional_derivative: .6f}")
    print(f"Directional Hessian:    {directional_hessian: .6f}\n")

# Define the functions
f_a = lambda x: x[0]**2 + x[1]**2
f_b = lambda x: (x[0] - 4)**2 + (x[1] - 2)**3
f_c = lambda x: np.sin(x[0]) + x[1] * x[2]**2

# Execute for each case
compute_directional_results(f_a, np.array([-2.0, 2.0]), np.array([1.0, -2.0]), "Function (a)")
compute_directional_results(f_b, np.array([4.0, 2.0]), np.array([1.0, -2.0]), "Function (b)")
compute_directional_results(f_c, np.array([1.0, 2.0, -1.0]), np.array([1.0, -2.0, 0.0]), "Function (c)")

--- Function (a) ---
Directional Derivative: -12.000000
Directional Hessian:     10.005375

--- Function (b) ---
Directional Derivative:  0.000000
Directional Hessian:     2.000001

--- Function (c) ---
Directional Derivative: -1.459698
Directional Hessian:    -0.841226



---
### **1.3**  
A common practive in programming is the developlemt of **unit tests**. A unit test is a method for making sure that a fcuntion does what you think it does, and is especially useful when prgoramming with AI. For this problem, we will develop unit tests for the 3rd-order interpolation scheme presented in Section 4.3.3, Equation 4.46.  
  
<center> First, download and install pytest following the getting started instructions on their website: docs.pytest.org     
</center>  
  
Second, create a folder named "test" and put it in your working vscode directory. All tests must be named with the "test_*.py" convention to be detected by vscode. For an example, I created "test_quadratic_roots.py" and place it within the test folder. 

In [ ]:
# test_quadratic_roots.py
import numpy as np
import pytest
def quadratic_roots(coefficients):
    coef = np.array(coefficients, dtype=float)
    a = coef[0]
    b = coef[1]
    c = coef[2]
    vertex =-b / (2*a)
    dist = np.sqrt(b**2- 4*a*c) / (2*a)
    root1 = vertex + dist
    root2 = vertex- dist
    return np.array([root1, root2])

def test_quadratic_produces_roots_1():
    # (x + 1)(x- 1) = x**2- 1
    coefficients = np.array([1, 0 ,-1])
    assert quadratic_roots(coefficients) == pytest.approx(np.array([1,-1]))

def test_quadratic_produces_roots_2():
    # (x + 3)(x + 1) = x**2 + 4x + 3
    coefficients = np.array([1, 4, 3])
    assert quadratic_roots(coefficients) == pytest.approx(np.array([-1,-3]))
    
def test_quadratic_produces_roots_3():
    # (x- 8)(x- 2) = x**2- 10x- 16
    coefficients = np.array([1,-10 , 16])
    assert quadratic_roots(coefficients) == pytest.approx(np.array([8, 2]))

Third, open the testing tab within vscode (the left side icon that looks like a test tube) and hit the button that says “configure python tests”. On the top of the screen it will ask you to pick between “unittest” and “pytest”, pick pytest as your testing package. The left sidebar should change, and there should be a black dot with the name of the directory you are working in. If you hover over the directory name, a run button should appear. This will run all the tests in the directory.  


Fourth, create a Python file outside the test folder and name it something like “line search utilities.py.” In this file make a function and call it “interpolate minimum alpha” and have it return 0. This function will be treated as a black box to our unit tests. Create a new test file in the test folder and name it “test line search utilities.py”. import numpy, pytest, and line search utilities. Create test functions to test the interpolation function, as well as helper functions to compute beta1 and beta2.

In [ ]:
# line_search_utilities.py

import numpy as np

def calculate_beta1(alpha_1,
                    alpha_2,
                    phi_at_alpha1,
                    phi_at_alpha2,
                    dphi_at_alpha1,
                    dphi_at_alpha2):
    return 0

def calculate_beta2(alpha_1,
                    alpha_2,
                    phi_at_alpha1,
                    phi_at_alpha2,
                    dphi_at_alpha1,
                    dphi_at_alpha2):
    return 0

def interpolate_minimum_alpha(alpha_1,
                            alpha_2,
                            phi_at_alpha1,
                            phi_at_alpha2,
                            dphi_at_alpha1,
                            dphi_at_alpha2):
    return 0


In [ ]:
# test_line_search_utilities.py

import numpy as np
import pytest
import hw2.line_search_utilities as lsu

def test_calculate_beta1_testcase1():
    assert False

def test_calculate_beta2_testcase1():
    assert False

def test_interpolate_minimum_alpha_testcase1():
    assert False

This is the part where you work out these values by hand. Pick values for α1 α2,ϕ(α1),ϕ(α2),ϕ′(α1),ϕ′(α2), and **work out by hand** what β1,β2, (for equations 4.47), and α∗ should be based on those inputs. We will provide one test case for you to test your code against, but we expect you to work out an additional test case and include it in the test file.

In [ ]:
# test_line_search_utilities.py

import numpy as np
import pytest
import hw2.line_search_utilities as lsu

def test_calculate_beta1_testcase1():
    alpha1 = 0.0
    alpha2 = 1.0
    phi_at_alpha1 = 1.0
    phi_at_alpha2 = 2.0
    dphi_at_alpha1 =-1.0
    dphi_at_alpha2 = 1.0
    assert lsu.calculate_beta1(alpha1,
                                alpha2,
                                phi_at_alpha1,
                                phi_at_alpha2,
                                dphi_at_alpha1,
                                dphi_at_alpha2) == pytest.approx(-3.0)

def test_calculate_beta2_testcase1():
    alpha1 = 0.0
    alpha2 = 1.0
    phi_at_alpha1 = 1.0
    phi_at_alpha2 = 2.0
    dphi_at_alpha1 =-1.0
    dphi_at_alpha2 = 1.0
    assert lsu.calculate_beta2(alpha1,
                                alpha2,
                                phi_at_alpha1,
                                phi_at_alpha2,
                                dphi_at_alpha1,
                                dphi_at_alpha2) == pytest.approx(np.sqrt(10))

def test_interpolate_minimum_alpha_testcase1():
    alpha1 = 0.0
    alpha2 = 1.0
    phi_at_alpha1 = 1.0
    phi_at_alpha2 = 2.0
    dphi_at_alpha1 =-1.0
    dphi_at_alpha2 = 1.0
    ans = 1- (4 + np.sqrt(10))/(2 + 2*np.sqrt(10))
    assert lsu.interpolate_minimum_alpha(alpha1,
                                        alpha2,
                                        phi_at_alpha1,
                                        phi_at_alpha2,
                                        dphi_at_alpha1,
                                        dphi_at_alpha2) == pytest.approx(ans)

Now that you have unit tests to verify that everything is working properly, you are encouraged to implement the calculate beta1, calculate beta2, and the interpolate minimum alpha functions with AI help if desired (but be careful not to miss out on learning...). To get points for this problem, submit your
test file with 3 additional test cases that you have personally worked out by hand, and if you used AI submit the AI prompt you used to implement the 3 helper functions. If you did not use AI, just write NO AI USED. While we will not force you to write unit tests for future assignments, it is **HIGHLY
ENCOURAGED** for debugging.

---
### **1.4**  
Develop a line search algorithm (i.e. Algorithm 4.3 and 4.4) that given a direction $p$, your algorithm returns the optimal point along that line (i.e. the optimal step length along $p$). Use the cubic interpolation method. Include your code in a zip folder. 

---
### **1.5**  
Test your line search algorithm with the following conditions. Report the optimal step length and point found ALONG $p$ (i.e. report both "$\alpha$" and $x_{(k=1)} = x_{(k=0)} + \alpha^*p$) and the number of function calls required to obtain $x_{(k=1)}$. (Remember this isn't the flobal minimum, just the minimum along the current $p$ direction.) It is important to note that the values you choose for $\mu_1$, $\mu_2$, $\alpha$, and the interpolation method will affect the outcome of the line search. For consistency with the answer key, use $\mu_1 = 1e-4$, $\mu_2 = 0.1$, $\alpha = 2.0$ for the inputs to the bracketing function, and use the cubic interpolation scheme within the pinpoint function. This interpolation scheme can be found at the end of section 4.3 in the textbook. Do not normalize $p$. Discuss what you learned with these tests in 200 words or less.  
  
(a) $p = [-1,1]^T, x_0 = [2,6]^T$, Slanted Quadratic Function wiht $\Beta = 1.5$ (See D.1.1 in Text Book)  
(b) $p = [1,-3]^T, x_0 = [0,2]^T$, Rosenbrock Function (See D.1.2 in Text Book)  
(c) $p = [1,2]^T, x_0 = [1,1]^T$, Jones Function (See D.1.4 in Text Book)